In [45]:
import yaml
import re
import nltk

class Token(object):
    def __init__(self, word):
        stemmer = nltk.stem.SnowballStemmer("english")
        self.token = stemmer.stem(word)
    
    @staticmethod
    def op_and(op1,op2):
        pass

    @staticmethod
    def op_or(op1,op2):
        pass

    @staticmethod
    def op_not(op):
        pass

    oprators_level = {')':0,'|':1,'&':2,'!':3,'(':4}
    operator_func = {'&': op_and.__func__, '|': op_or.__func__, '!': op_not.__func__}
        

In [27]:
origin_query_str = "A       And    B    And ( not C or  D aNd(e oR F))"
query_str = origin_query_str.lower().replace("("," ( ").replace(")"," ) ").replace("and",'&').replace("or",'|').replace("not",'!')
query_exp = query_str.split()

In [47]:
oprand_stack = list()
oprator_stack = list()
for element in query_exp:
    if element in Token.oprators_level:
        top = oprator_stack[-1]
        if element == '(' or Token.oprators_level[element] > Token.oprators_level[top]:
            oprator_stack.append(element)
        elif element == ')':
            while True:
                oprator = oprator_stack.pop()
                if oprator == '(':
                    break
                elif oprator == '!':
                    oprand = oprand_stack.pop()
                    result = Token.oprator_func[oprator](oprand)
                else:
                    oprand1 = oprand_stack.pop()
                    oprand2 = oprand_stack.pop()
                    result = Token.oprator_func[oprator](oprand1,oprand2)
                oprand_stack.append(result)
        else:
            while True:
                oprator = oprator_stack.pop()
                if Token.oprators_level[element] >Token. oprators_level[oprator] or oprator == '(':
                    oprator_stack.append(oprator)
                    oprator_stack.append(element)
                    break
                elif oprator == '!':
                    oprand = oprand_stack.pop()
                    result = Token.oprator_func[oprator](oprand)
                else:
                    oprand1 = oprand_stack.pop()
                    oprand2 = oprand_stack.pop()
                    result = Token.oprator_func[oprator](oprand1,oprand2)
                oprand_stack.append(result)
    else:
        oprand = Token(element)
        oprand_stack.append(oprand)

IndexError: list index out of range